In [112]:
import re

# Part 1

In [146]:
file = open('Inputs/input_day12.txt', 'r')


transcript = re.sub("[^a-z0-9A-Z'-]", ' ', file.read())
transcript = re.sub(" +", " ", transcript)
numbers = [int(n) for n in transcript.split() if n.lstrip("-").isdigit()]


print(f"FINAL ANSWER: {sum(numbers)}")

FINAL ANSWER: 191164


# Part 2

In [155]:
file = open('Inputs/input_day12.txt', 'r')
text = file.read()


def calculate_numbers(phrase):
    transcript = re.sub("[^a-z0-9A-Z'-]", ' ', phrase)
    transcript = re.sub(" +", " ", transcript)
    numbers = [int(n) for n in transcript.split() if n.lstrip("-").isdigit()]
    return sum(numbers)


def get_inds_lvls(text):
    lvl = 0
    lvls = [0]
    inds = []
    to_remove = 0
    for i, char in enumerate(text):
        if char == "{":
            lvl += 1
            inds.append(i)
            lvls.append(lvl)
        elif char == "}":
            lvl -= 1
            inds.append(i)
            lvls.append(lvl)
    return inds, lvls

    
class Node(object):
    
    def __init__(self, phrase, depth, inds):
        self.phrase = phrase
        self.depth = depth
        self.inds = inds
        self.children = []
        self.seen = False
        
    
    def list_children(self):
        for child in self.children:
            print(child.phrase)
            
    
    def calculate_number(self):
        return calculate_numbers(self.phrase)
    
    
    def add_parent(self, node):
        node.children = [self] + node.children
        
    
    def get_clean_text(self):
        relevant = [self.inds[0]]
        for c in self.children:
            relevant += c.inds 
        relevant += [self.inds[1]]
        relevant_text = ""
        for i in range(0, len(relevant), 2):
            relevant_text += text_example[relevant[i]:relevant[i+1]+1]
        return relevant_text
        
        
class PhraseTree(object):
    
    def __init__(self, text, inds, lvls):
        self.text = text
        self.inds = inds
        self.lvls = lvls
        self.root_nodes = []
        
    
    def create_tree(self):
        aux = []
        newest_nodes = []
        for i, lvl in enumerate(self.lvls):
            if lvl > 0 and self.lvls[i-1] < lvl:
                aux.append(self.inds[i-1])
            elif self.lvls[i-1] > lvl:
                phrase = self.text[aux[-1]:self.inds[i-1]+1]
                node = Node(phrase, self.lvls[i-1], [aux[-1], self.inds[i-1]])
                while newest_nodes != [] and newest_nodes[-1].depth > self.lvls[i-1]:
                    newest_nodes[-1].add_parent(node)
                    newest_nodes.pop()
                newest_nodes.append(node)
                aux.pop()
                if node.depth == 1:
                    self.root_nodes.append(node)

                    
                    
text_example = "pppp{qqqqqqq{rrrr}qq{sss}qqqq}ppp"
inds, lvls = get_inds_lvls(text)
pt = PhraseTree(text, inds, lvls)
pt.create_tree()


to_remove = 0
for i in range(len(pt.root_nodes)):
    root = pt.root_nodes[i]
    levels = [root]
    while levels != []:
        current = levels[-1]
        if current.children == []:
            occurences = [(m.start(), m.end()) for m in re.finditer('"red"', current.phrase)]
            if occurences != [] and not current.seen and any(list(map(lambda p: current.phrase[p[0]-1] == ':' or current.phrase[p[1]] == ':', occurences))):
                to_remove += calculate_numbers(current.phrase)
            current.seen = True
            levels.pop()
        else:
            clean_text = current.get_clean_text()
            occurences = [(m.start(), m.end()) for m in re.finditer('"red"', clean_text)]
            if occurences != [] and not current.seen: # and any(list(map(lambda p: clean_text[p[0]-1] == ':' or clean_text[p[1]] == ':', occurences))): 
                to_remove += calculate_numbers(current.phrase)
                levels.pop()
            else:
                new = current.children[-1]
                current.children.pop()
                levels.append(new)
            current.seen = True

            

print(f"FINAL ANSWER: {sum(numbers) - to_remove}")

FINAL ANSWER: 165071


In [156]:
import json

def day12():
    def sum_numbers(obj):
        if type(obj) == type(dict()):
            if "red" in obj.values():
                return 0
            return sum(map(sum_numbers, obj.values()))

        if type(obj) == type(list()):
            return sum(map(sum_numbers, obj))

        if type(obj) == type(0):
            return obj

        return 0

    data = json.loads(text)
    return sum_numbers(data)

day12()

87842